# Selecting Subsets of Data in Pandas

This notebook is also available as a [blog post on Medium](https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-39e811c81a0c).

## Part 2: Boolean Indexing
This is part 2 of a seven-part series on how to select subsets of data from a pandas DataFrame or Series. Pandas offers a wide variety of options for subset selection which necessitates multiple articles. This series is broken down into the following 7 topics.

1. Selection with `[]`, `.loc` and `.iloc`
1. Boolean indexing
1. Assigning subsets of data
1. Selection with a MultiIndex
1. Selecting subsets of data with methods
1. Selections with other Index types
1. Internals, Miscellaneous, and Conclusion

## Part 1 vs Part 2 subset selection

Part 1 of this series covered subset selection with `[]`, `.loc` and `.iloc`. All three of these **indexers** use either the row/column labels or their integer location to make selections. The actual **data** of the Series/DataFrame is not used at all during the selection. 

In Part 2 of this series, on **boolean indexing**, we will select subsets of data based on the actual values of the data in the Series/DataFrame and NOT on their row/column labels or integer locations. 


## Documentation on boolean selection
I will always recommend reading the official documentation in addition to this tutorial when learning about boolean selection. The documentation uses more formal examples with dummy data, but is still an excellent resource.

The documentation use the term **boolean indexing** but you will also see **boolean selection**.

[Boolean Indexing from pandas documentation](http://pandas.pydata.org/pandas-docs/stable/indexing.html#boolean-indexing)

## Stack Overflow Data

The data that we will use for this tutorial comes from [Stack Overflow's data explorer](https://data.stackexchange.com/stackoverflow/query/new), which is a fantastic tool to gather an incredible amount of data from the site. You must know SQL in order to use the data explorer. The data explorer allows you to save queries. [Take a look at the query](http://data.stackexchange.com/stackoverflow/query/768430/get-all-questions-and-answerers-from-tag) I used to collect the data. 

The table below contains data on each question asked on stack overflow [tagged as pandas](https://stackoverflow.com/questions/tagged/pandas). 

The first question was asked March 30, 2011. Since then, more than 56,000 questions have been added as of December 2, 2017.

In [1]:
import pandas as pd
import numpy as np

In [2]:
so = pd.read_csv('../../data/stackoverflow_qa.csv')
so.head()

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
0,5486226,2011-03-30 12:26:50,4,2113,Rolling median in python,3,4,1.0,yueerhu,125.0,Mike Pennington,26995.0
1,5515021,2011-04-01 14:50:44,8,7015,Compute a compounded return series in Python,3,6,7.0,Jason Strimpel,3301.0,Mike Pennington,26995.0
2,5558607,2011-04-05 21:13:50,2,7392,Sort a pandas DataMatrix in ascending order,2,0,1.0,Jason Strimpel,3301.0,Wes McKinney,43310.0
3,6467832,2011-06-24 12:31:45,9,13056,How to get the correlation between two timeser...,1,0,7.0,user814005,117.0,Wes McKinney,43310.0
4,7577546,2011-09-28 01:58:38,9,2488,"Using pandas, how do I subsample a large DataF...",1,0,5.0,Uri Laserson,958.0,HYRY,54137.0


## Asking simple questions in plain English

Before we get to the technical definition of boolean indexing, let's see some examples of the types of questions it can answer.

* Find all questions that were created before 2014
* Find all questions with a score more than 50
* Find all questions with a score between 50 and 100
* Find all questions answered by Scott Boston
* Find all questions answered by the following 5 users
* Find all questions that were created between March, 2014 and October 2014 that were answered by Unutbu and have score less than 5.
* Find all questions that have score between 5 and 10 or have a view count of greater than 10,000
* Find all questions that are not answered by Scott Boston

You will also see examples like this referred to by the term **queries**.

## All queries have criteria
Each of the above queries have a strict logical criteria that must be checked one row at a time.

## Keep or Discard entire row of data
If you were to manually answer the above queries, you would need to scan each row and determine whether the row as a whole meets the criterion or not. If the row meets the criteria, then it is kept and if not, then it is discarded.

## Each row will have a `True` or `False` value associated with it
When you perform boolean indexing, each row of the DataFrame (or value of a Series) will have a `True` or `False` value associated with it depending on whether or not it meets the criterion. True/False values are known as **boolean**. The documentation refers to the entire procedure as **boolean indexing**. 

Since we are using the booleans to select data, it is sometimes referred to as **boolean selection**. Essentially, we are using booleans to select subsets of data.

## Using `[]` and `.loc` for boolean selection
We will use the same three indexers, **`[]`** and **`.loc`** from part 1 to complete our boolean selections. We will do so by placing a sequence of booleans inside of these indexer. The sequence will be the same number of rows/values as the DataFrame/Series it is doing the selection on.

The **`.iloc`** indexer can be made to work with boolean selection but is almost never used. A small section towards the end will show why it's unnecessary.

## Focus on `[]` for now
To simplify things, we will only the brackets, **`[]`**, which I called **just the indexing operator** from part 1. We will get to the other indexers a bit later.

## Use a small DataFrame to get started
Before we make our first boolean selection, let's simplify matters and use the first five rows of the stack overflow data as our starting DataFrame.

In [3]:
so_head = so.head()
so_head

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
0,5486226,2011-03-30 12:26:50,4,2113,Rolling median in python,3,4,1.0,yueerhu,125.0,Mike Pennington,26995.0
1,5515021,2011-04-01 14:50:44,8,7015,Compute a compounded return series in Python,3,6,7.0,Jason Strimpel,3301.0,Mike Pennington,26995.0
2,5558607,2011-04-05 21:13:50,2,7392,Sort a pandas DataMatrix in ascending order,2,0,1.0,Jason Strimpel,3301.0,Wes McKinney,43310.0
3,6467832,2011-06-24 12:31:45,9,13056,How to get the correlation between two timeser...,1,0,7.0,user814005,117.0,Wes McKinney,43310.0
4,7577546,2011-09-28 01:58:38,9,2488,"Using pandas, how do I subsample a large DataF...",1,0,5.0,Uri Laserson,958.0,HYRY,54137.0


## Manually create a list of booleans
For our first boolean selection, we will not answer any interesting 'English' queries and instead just select rows with a list of booleans.

For instance, let's select the first and third rows by creating the following list:

In [4]:
criteria = [True, False, True, False, False]

We can pass this list of booleans to just the indexing operator and complete our selection:

In [5]:
so_head[criteria]

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
0,5486226,2011-03-30 12:26:50,4,2113,Rolling median in python,3,4,1.0,yueerhu,125.0,Mike Pennington,26995.0
2,5558607,2011-04-05 21:13:50,2,7392,Sort a pandas DataMatrix in ascending order,2,0,1.0,Jason Strimpel,3301.0,Wes McKinney,43310.0


## Wait a second... Isn't `[]` just for column selection?
The primary purpose of *just the indexing operator* for a DataFrame is to select one or more columns by using either a string or a list of strings. Now, all of a sudden, this example is showing that entire rows are selected with boolean values. This is what makes pandas, unfortunately, one of the most confusing libraries to use. 

## Operator Overloading
*Just the indexing* operator is overloaded. This means, that depending on the inputs, pandas will do something completely different. Here are the rules for the different objects you pass to *just the indexing operator*.
* string - return a column as a Series
* list of strings - return all those columns as a DataFrame
* a slice - select rows (can do both label and integer location - confusing!)
* a sequence of booleans - select all rows where **`True`**

In summary, primarily *just the indexing operator* selects **columns**, but if you pass it a sequence of booleans it will select all **rows** that are **`True`**.

## What do you mean by 'sequence'?
I keep using the term **sequence of booleans** to refer to the `True/False` values. Technically, the most common built-in [Python sequence](https://docs.python.org/3/library/stdtypes.html#typesseq) types are lists and tuples. In addition to a list, you will most often be using a pandas Series as your 'sequence' of booleans.

Let's manually create a boolean Series to select the last three rows of **`so_head`**.

In [6]:
s = pd.Series([False, False, True, True, True])
s

0    False
1    False
2     True
3     True
4     True
dtype: bool

In [7]:
so_head[s]

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
2,5558607,2011-04-05 21:13:50,2,7392,Sort a pandas DataMatrix in ascending order,2,0,1.0,Jason Strimpel,3301.0,Wes McKinney,43310.0
3,6467832,2011-06-24 12:31:45,9,13056,How to get the correlation between two timeser...,1,0,7.0,user814005,117.0,Wes McKinney,43310.0
4,7577546,2011-09-28 01:58:38,9,2488,"Using pandas, how do I subsample a large DataF...",1,0,5.0,Uri Laserson,958.0,HYRY,54137.0


## Take care when creating a boolean Series by hand
The above example only worked because the index of both the boolean Series and **`so_head`** were the exact same. Let's output them so you can clearly see this.

In [8]:
s.index

RangeIndex(start=0, stop=5, step=1)

In [9]:
so_head.index

RangeIndex(start=0, stop=5, step=1)

## Boolean selection fails when the index doesn't align
When you are using a boolean Series to do boolean selection, the index of both objects must be the exact same. Let's create a slightly different Series with a different index than the DataFrame it is indexing on.

In [10]:
s = pd.Series([False, False, True, True, True], index=[2, 3, 4, 5, 6])
s

2    False
3    False
4     True
5     True
6     True
dtype: bool

In [11]:
so_head[s]

/Users/Ted/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

## `IndexingError`: Unalignable boolean Series!
If the index of both the boolean Series and the object you are doing boolean selection on don't match exactly, you will get the above error. This is one reason, as you will below, why you will almost never create boolean Series by hand like this.

## Also use NumPy arrays
You can also use NumPy arrays to do boolean selection. NumPy arrays have no index so you won't get the error above, but your array needs to be the same exact length as the object you are doing boolean selection on.

In [12]:
a = np.array([True, False, False, False, False])
so_head[a]

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
0,5486226,2011-03-30 12:26:50,4,2113,Rolling median in python,3,4,1.0,yueerhu,125.0,Mike Pennington,26995.0


## Never creating boolean Series by hand
You will likely never create a boolean Series by hand as was done above. Instead, you will produce them based on the values of your data.

## Use the comparison operators to create boolean Series
The primary method of creating a Series of booleans is to use one of the six comparison operators: 
* **`<`**
* **`<=`**
* **`>`**
* **`>=`**
* **`==`**
* **`!=`** 

## Use comparison operator with a single column of data
You will almost always use the comparison operators on just a single column or Series of data. For instance, let's create a boolean Series from the **`score`** column. Let's determine if the score is at least 10.

We select the score column and then test the condition that each value is greater than or equal to 10. Notice that this operations gets applied to each value in the Series. A boolean Series is returned.

In [13]:
criteria = so['score'] >= 10
criteria.head(10)

0    False
1    False
2    False
3    False
4    False
5    False
6     True
7     True
8     True
9    False
Name: score, dtype: bool

## Finally making a boolean selection
Now that we have our boolean Series stored in the variable **`criteria`**, we can pass this to *just the indexing operator* to select only the rows that have a score of at least 10. 

We are going to use the entire **`so`** DataFrame for the rest of the tutorial.

In [14]:
so_score_10_or_more = so[criteria]
so_score_10_or_more.head()

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
6,7776679,2011-10-15 08:21:17,25,28159,append two data frame with pandas,2,7,4.0,Jean-Pat,882.0,Wes McKinney,43310.0
7,7813132,2011-10-18 20:16:12,10,18917,Convert array of string (category) to array of...,3,0,6.0,Jean-Pat,882.0,Wes McKinney,43310.0
8,7837722,2011-10-20 14:46:14,201,223746,What is the most efficient way to loop through...,8,3,115.0,Muppet,1563.0,Nick Crawford,2779.0
14,8916302,2012-01-18 19:41:27,29,20614,selecting across multiple columns with python ...,3,0,14.0,user248237dfsf,19244.0,Wes McKinney,43310.0
17,8991709,2012-01-24 17:59:53,136,16783,Why are pandas merges in python faster than da...,3,16,60.0,Zach,12484.0,Matt Dowle,41275.0


## How many rows have a score of at least ten
Just by looking at the head of the resulting DataFrame, we don't know how many rows passed our criterion. Let's output the shape of both our original and our resulting DataFrame.

In [15]:
so.shape

(56398, 12)

In [16]:
so_score_10_or_more.shape

(1505, 12)

Only about 3% of questions get a score of 10 or more.

## Boolean selection in one line
Often, you will see boolean selection happen in a single line of code instead of the multiple lines we used above. If the following is confusing for you, then I recommend storing your boolean Series to a variable like I did with **`criteria`** above.

It is possible to put the creation of the boolean Series inside of *just the indexing operator* like this.

In [17]:
so[so['score'] >= 10].head()

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
6,7776679,2011-10-15 08:21:17,25,28159,append two data frame with pandas,2,7,4.0,Jean-Pat,882.0,Wes McKinney,43310.0
7,7813132,2011-10-18 20:16:12,10,18917,Convert array of string (category) to array of...,3,0,6.0,Jean-Pat,882.0,Wes McKinney,43310.0
8,7837722,2011-10-20 14:46:14,201,223746,What is the most efficient way to loop through...,8,3,115.0,Muppet,1563.0,Nick Crawford,2779.0
14,8916302,2012-01-18 19:41:27,29,20614,selecting across multiple columns with python ...,3,0,14.0,user248237dfsf,19244.0,Wes McKinney,43310.0
17,8991709,2012-01-24 17:59:53,136,16783,Why are pandas merges in python faster than da...,3,16,60.0,Zach,12484.0,Matt Dowle,41275.0


## Single condition expression
Our first example tested a single condition (whether the score was 10 or more). Let's test a different single condition and look for all the questions that are answered by **Scott Boston**. The **`ans_name`** variable holds the display names of the people who posted the accepted answer to the question.

We use the **`==`** operator to test for equality and again store this result to the variable **`criteria`**. Again, we pass this variable to *just the indexing operator* which completes our selection.

In [18]:
# step 1 - create boolean Series
criteria = so['ans_name'] == 'Scott Boston'

# step 2 - do boolean selection
so[criteria].head()

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
38161,43491342,2017-04-19 09:14:28,4,167,Merging pandas dataframes based on nearest val...,1,0,NaN,AkiRoss,3991.0,Scott Boston,23611.0
38178,43190850,2017-04-03 17:31:33,1,284,Python Seaborn Plot ValueError,2,3,NaN,Ryan,545.0,Scott Boston,23611.0
38237,43176052,2017-04-03 03:21:12,2,39,Convert an indexed pandas matrix to a flat dat...,2,0,NaN,alvas,31923.0,Scott Boston,23611.0
38246,43209525,2017-04-04 14:03:17,5,131,Pandas: Optimal way to MultiIndex columns,2,0,0.0,sparc_spread,5470.0,Scott Boston,23611.0
38275,43211893,2017-04-04 15:45:17,0,38,How to calculate a index series for a event wi...,1,3,NaN,zsljulius,1102.0,Scott Boston,23611.0


## Multiple condition expression
So far, both our boolean selections have involved a single condition. You can, of course, have as many conditions as you would like. To do so, you will need to combine your boolean expressions using the three logical operators **and**, **or** and **not**.

## Use `&`, `|`, `~`
Although Python uses the syntax **`and`**, **`or`**, and **`not`**, these will not work when testing multiple conditions with pandas. The details of why this is so will be left for part 7 of the Series.

You must use the following operators with pandas:
* **`&`** for **and**
* **`|`** for **or**
* **`~`** for **not**

## Our first multiple condition expression
Let's find all the questions that have a score of at least 5 and are answered by Scott Boston. To begin, we will create two separate variable to hold each criteria.

In [19]:
criteria_1 = so['score'] >= 5
criteria_2 = so['ans_name'] == 'Scott Boston'

We will then use the **and** operator, the ampersand **`&`**, to combine them

In [20]:
criteria_all = criteria_1 & criteria_2

We can now pass this final criteria to *just the indexing operator*

In [21]:
so[criteria_all]

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
38246,43209525,2017-04-04 14:03:17,5,131,Pandas: Optimal way to MultiIndex columns,2,0,0.0,sparc_spread,5470.0,Scott Boston,23611.0
38640,42870703,2017-03-18 05:06:22,5,125,Simultaneous operation of groupby and resample...,1,0,1.0,S. Naribole,43.0,Scott Boston,23611.0
44358,45064916,2017-07-12 18:16:49,5,428,How to find the correlation between a group of...,2,5,NaN,BKS,506.0,Scott Boston,23611.0
44814,44877663,2017-07-03 04:07:59,9,1267,Error: float object has no attribute notnull,3,2,1.0,Vivian Tio,181.0,Scott Boston,23611.0
52013,47061564,2017-11-01 18:40:36,5,60,How to create strings from dataframe columns e...,5,0,NaN,hernanavella,1890.0,Scott Boston,23611.0


## Multiple conditions in one line
It is possible to combine the entire expression into a single line. Many pandas users like doing this, others hate it. Regardless, it is a good idea to know how to do so as you will definitely encounter it.

## Use parentheses to separate conditions
You must encapsulate each condition in a set of parentheses in order to make this work. This again, will be explained in part 7.

Each condition will be separated like this:
```Python
(so['score'] >= 5) & (so['ans_name'] == 'Scott Boston')
```

We can then drop this expression inside of *just the indexing operator*

In [22]:
so[(so['score'] >= 5) & (so['ans_name'] == 'Scott Boston')]

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
38246,43209525,2017-04-04 14:03:17,5,131,Pandas: Optimal way to MultiIndex columns,2,0,0.0,sparc_spread,5470.0,Scott Boston,23611.0
38640,42870703,2017-03-18 05:06:22,5,125,Simultaneous operation of groupby and resample...,1,0,1.0,S. Naribole,43.0,Scott Boston,23611.0
44358,45064916,2017-07-12 18:16:49,5,428,How to find the correlation between a group of...,2,5,NaN,BKS,506.0,Scott Boston,23611.0
44814,44877663,2017-07-03 04:07:59,9,1267,Error: float object has no attribute notnull,3,2,1.0,Vivian Tio,181.0,Scott Boston,23611.0
52013,47061564,2017-11-01 18:40:36,5,60,How to create strings from dataframe columns e...,5,0,NaN,hernanavella,1890.0,Scott Boston,23611.0


## Using an `or` condition
Let's find all the questions that have a score of at least 100 or have at least 10 answers.

For the **or** condition, we use the pipe **`|`**

In [23]:
so[(so['score'] >= 100) | (so['answercount'] >= 10)].head()

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
8,7837722,2011-10-20 14:46:14,201,223746,What is the most efficient way to loop through...,8,3,115.0,Muppet,1563.0,Nick Crawford,2779.0
17,8991709,2012-01-24 17:59:53,136,16783,Why are pandas merges in python faster than da...,3,16,60.0,Zach,12484.0,Matt Dowle,41275.0
60,10065051,2012-04-08 18:01:13,83,77902,python-pandas and databases like mysql,11,4,65.0,user1320615,532.0,NaN,NaN
75,10373660,2012-04-29 16:10:35,199,207980,Converting a Pandas GroupBy object to DataFrame,5,0,90.0,saveenr,2421.0,Wes McKinney,43310.0
99,10636024,2012-05-17 12:48:00,36,30386,Python / Pandas - GUI for viewing a DataFrame ...,15,6,27.0,Ross R,1562.0,user1319128,555.0


## Reversing a condition with the `not` operator
The tilde character **`~`** represents the **not** operator and reverses a condition. For instance, if we wanted all the questions with score greater than 100, we could do it like this:

In [24]:
so[~(so['score'] <= 100)].head()

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
8,7837722,2011-10-20 14:46:14,201,223746,What is the most efficient way to loop through...,8,3,115.0,Muppet,1563.0,Nick Crawford,2779.0
17,8991709,2012-01-24 17:59:53,136,16783,Why are pandas merges in python faster than da...,3,16,60.0,Zach,12484.0,Matt Dowle,41275.0
75,10373660,2012-04-29 16:10:35,199,207980,Converting a Pandas GroupBy object to DataFrame,5,0,90.0,saveenr,2421.0,Wes McKinney,43310.0
100,10665889,2012-05-19 14:11:42,144,179896,How to take column-slices of dataframe in pandas,7,3,65.0,cpa,988.0,Ted Petrou,10426.0
106,10715965,2012-05-23 08:12:31,340,408347,add one row in a pandas.DataFrame,15,3,89.0,PhE,1988.0,fred,2342.0


Notice that there were parentheses around the condition '**`score`** less than equal to 100'. We had to use parentheses here or the operation wouldn't work correctly. 

Of course, this trivial example has no need for the not operator and can be replaced with the greater than operator, but it's easy to verify.

Let's look back up one example and invert the condition of **`score`** at least 100 or number of answers at least 10.  To do this, we will have to wrap our entire expression with parentheses like this:

```Python
~((so['score'] >= 100) | (so['answercount'] >= 10))
```

There is a set of parentheses around each inner expression as well.

## Complex conditions
It is possible to build extremely complex conditions to select rows of your DataFrame that meet a very specific criteria. For instance, we can select all questions answered by Scott Boston with **`score`** 5 or more OR questions answered by Ted Petrou with answer count 5 or more.

With multiple conditions, its probably best to break out the logic into multiple steps:

In [25]:
criteria_1 = (so['score'] >= 5) & (so['ans_name'] == 'Scott Boston')
criteria_2 = (so['answercount'] >= 5) & (so['ans_name'] == 'Ted Petrou')
criteria_all = criteria_1 | criteria_2
so[criteria_all]

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
100,10665889,2012-05-19 14:11:42,144,179896,How to take column-slices of dataframe in pandas,7,3,65.0,cpa,988.0,Ted Petrou,10426.0
38246,43209525,2017-04-04 14:03:17,5,131,Pandas: Optimal way to MultiIndex columns,2,0,0.0,sparc_spread,5470.0,Scott Boston,23611.0
38640,42870703,2017-03-18 05:06:22,5,125,Simultaneous operation of groupby and resample...,1,0,1.0,S. Naribole,43.0,Scott Boston,23611.0
44358,45064916,2017-07-12 18:16:49,5,428,How to find the correlation between a group of...,2,5,NaN,BKS,506.0,Scott Boston,23611.0
44814,44877663,2017-07-03 04:07:59,9,1267,Error: float object has no attribute notnull,3,2,1.0,Vivian Tio,181.0,Scott Boston,23611.0
52013,47061564,2017-11-01 18:40:36,5,60,How to create strings from dataframe columns e...,5,0,NaN,hernanavella,1890.0,Scott Boston,23611.0


## Lots of `or` conditions in a single column - use `isin`
Occasionally, we will want to test equality in a single column to multiple values. This is most common in string columns. For instance, let's say we wanted to find all the questions answered by Scott Boston, Ted Petrou, MaxU, and unutbu.

One way to do this would be with four `or` conditions.

```Python
criteria = ((so['ans_name'] == 'Scott Boston') | (so['ans_name'] == 'Ted Petrou') | 
            (so['ans_name'] == 'MaxU') | (so['ans_name'] == 'unutbu'))
```

An easier way is to use the Series method **`isin`**. Pass it a list of all the items you want to check for equality.

In [26]:
criteria = so['ans_name'].isin(['Scott Boston', 'Ted Petrou', 'MaxU', 'unutbu'])
criteria.head()

0    False
1    False
2    False
3    False
4    False
Name: ans_name, dtype: bool

In [27]:
so[criteria].head()

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
100,10665889,2012-05-19 14:11:42,144,179896,How to take column-slices of dataframe in pandas,7,3,65.0,cpa,988.0,Ted Petrou,10426.0
574,13385860,2012-11-14 19:25:28,29,31712,How can I remove extra whitespace from strings...,6,0,17.0,mpjan,530.0,unutbu,464745.0
593,13446480,2012-11-19 01:20:07,18,7969,Python Pandas: remove entries based on the num...,4,1,7.0,sashkello,8714.0,unutbu,464745.0
599,13460889,2012-11-19 19:39:06,8,3168,How to redirect all methods of a contained cla...,2,4,5.0,Yariv,5013.0,unutbu,464745.0
639,13653030,2012-11-30 20:54:35,6,11798,How do I Pass a List of Series to a Pandas Dat...,5,1,1.0,rhaskett,323.0,unutbu,464745.0


## Combining `isin` with other criteria
You can use the resulting boolean Series from the **`isin`** method in the same way you would from the logical operators. For instance, If we wanted to find all the questions answered by the people above and had score greater than 30 we would do the following:

In [28]:
criteria_1 = so['ans_name'].isin(['Scott Boston', 'Ted Petrou', 'MaxU', 'unutbu'])
criteria_2 = so['score'] > 30
criteria_all = criteria_1 & criteria_2
so[criteria_all].tail()

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
4389,21800169,2014-02-15 16:18:11,81,183564,Python Pandas: Get index of rows which column ...,2,0,40.0,I want badges,708.0,unutbu,464745.0
5179,21415661,2014-01-28 20:04:04,38,46056,Logic operator for boolean indexing in Pandas,1,3,16.0,user2988577,647.0,unutbu,464745.0
7411,25254016,2014-08-11 23:30:16,73,113632,Pandas - Get first row value of a given column,4,1,10.0,Ahmed Haque,1363.0,unutbu,464745.0
11136,29370057,2015-03-31 13:38:06,51,72821,Select dataframe rows between two dates,4,0,26.0,darkpool,1502.0,unutbu,464745.0
25710,38250710,2016-07-07 16:26:26,47,20477,"How to split data into 3 sets (train, validati...",3,7,15.0,CentAu,1923.0,MaxU,83732.0


## Use `isnull` to find rows with missing values
The **`isnull`** method returns a boolean Series where True indicates where a missing value is. For instance, questions that do not have an **accepted answer** have missing values for **`ans_name`**. Let's call **`isnull`** on this column.

In [29]:
no_answer = so['ans_name'].isnull()
no_answer.head(6)

0    False
1    False
2    False
3    False
4    False
5     True
Name: ans_name, dtype: bool

This is just another boolean Series which we can pass to *just the indexing operator*

In [30]:
so[no_answer].head()

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
5,7766400,2011-10-14 10:33:54,2,977,Pandas + Django + mod_wsgi + virtualenv,1,2,1.0,Evan Davey,100.0,NaN,NaN
11,8451327,2011-12-09 20:27:24,1,2435,Python map() function output into Pandas DataF...,1,0,1.0,briant57,6.0,NaN,NaN
16,8966871,2012-01-23 03:21:00,0,287,Running Python/Numpy/Pandas on older secure co...,0,3,NaN,Casey,16.0,NaN,NaN
29,9641916,2012-03-09 22:36:52,7,5519,Python Pandas: can't find numpy.core.multiarra...,4,2,NaN,Dylan Cutler,41.0,NaN,NaN
30,9647656,2012-03-10 15:35:26,3,986,Pandas dataframe in mixed mode can't serialize...,1,0,NaN,David van Coevorden,45.0,NaN,NaN


An alias of **`isnull`** is the **`isna`** method. Alias means it is the same exact method with a different name.

# Boolean Selection on a Series
All the examples thus far have taken place on the **`so`** DataFrame. Boolean selection on a Series happens almost identically. Since there is only one dimension of data, the queries you ask are usually going to be simpler.

First, let's select a single column of data as a Series such as the **`commentcount`** column.

In [31]:
s = so['commentcount']
s.head()

0    4
1    6
2    0
3    0
4    0
Name: commentcount, dtype: int64

Let's test for number of comments greater than 10

In [32]:
criteria = s > 10
criteria.head()

0    False
1    False
2    False
3    False
4    False
Name: commentcount, dtype: bool

Notice that there is no column selection here as we are already down to a single column. Let's pass this criteria to *just the indexing operator* to select just the values greater than 10.

In [33]:
s[criteria].head()

17     16
76     14
566    11
763    12
781    19
Name: commentcount, dtype: int64

We could have done this in one step like this

In [34]:
s[s > 10].head()

17     16
76     14
566    11
763    12
781    19
Name: commentcount, dtype: int64

If we wanted to find those comments greater than 10 but less than 15 we could have used an **and** condition like this:

In [35]:
s[(s > 10) & (s < 15)].head()

76     14
566    11
763    12
787    12
837    13
Name: commentcount, dtype: int64

## Another possibility is the `between` method
Pandas has lots of duplicate functionality built in to it. Instead of writing two boolean conditions to select all values inside of a range as was done above, you can use the **`between`** method to create a boolean Series. To use, pass it the left and right end points of the range. These endpoints are inclusive.

So, to replicate the previous example, you could have done this:

In [36]:
s[s.between(11, 14)].head()

76     14
566    11
763    12
787    12
837    13
Name: commentcount, dtype: int64

# Simultaneous boolean selection with rows and column labels with `.loc`
The **`.loc`** indexer was thoroughly covered in part 1 and will now be covered here to simultaneously select rows and columns. In part 1, it was stated that **`.loc`** made selections only by **label**. This wasn't strictly true as it is also able to do boolean selection along with selection by label.

Remember that **`.loc`** takes both a row selection and a column selection separated by a comma. Since the row selection comes first, you can pass it the same exact inputs that you do for *just the indexing operator* and get the same results.

Let's take a look at a couple examples from above:

In [37]:
# same as above
so.loc[(so['score'] >= 5) & (so['ans_name'] == 'Scott Boston')]

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
38246,43209525,2017-04-04 14:03:17,5,131,Pandas: Optimal way to MultiIndex columns,2,0,0.0,sparc_spread,5470.0,Scott Boston,23611.0
38640,42870703,2017-03-18 05:06:22,5,125,Simultaneous operation of groupby and resample...,1,0,1.0,S. Naribole,43.0,Scott Boston,23611.0
44358,45064916,2017-07-12 18:16:49,5,428,How to find the correlation between a group of...,2,5,NaN,BKS,506.0,Scott Boston,23611.0
44814,44877663,2017-07-03 04:07:59,9,1267,Error: float object has no attribute notnull,3,2,1.0,Vivian Tio,181.0,Scott Boston,23611.0
52013,47061564,2017-11-01 18:40:36,5,60,How to create strings from dataframe columns e...,5,0,NaN,hernanavella,1890.0,Scott Boston,23611.0


In [38]:
# same as above
criteria = so['ans_name'].isin(['Scott Boston', 'Ted Petrou', 'MaxU', 'unutbu'])
so.loc[criteria].head()

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
100,10665889,2012-05-19 14:11:42,144,179896,How to take column-slices of dataframe in pandas,7,3,65.0,cpa,988.0,Ted Petrou,10426.0
574,13385860,2012-11-14 19:25:28,29,31712,How can I remove extra whitespace from strings...,6,0,17.0,mpjan,530.0,unutbu,464745.0
593,13446480,2012-11-19 01:20:07,18,7969,Python Pandas: remove entries based on the num...,4,1,7.0,sashkello,8714.0,unutbu,464745.0
599,13460889,2012-11-19 19:39:06,8,3168,How to redirect all methods of a contained cla...,2,4,5.0,Yariv,5013.0,unutbu,464745.0
639,13653030,2012-11-30 20:54:35,6,11798,How do I Pass a List of Series to a Pandas Dat...,5,1,1.0,rhaskett,323.0,unutbu,464745.0


## Separate row and column selection with a comma for `.loc`
The great benefit of **`.loc`** is that it allows you to simultaneously do boolean selection along the rows and make column selections by label.

For instance, let's say we wanted to find all the questions with more than 20k views but only return the **`creationdate`**, **`viewcount`**, and **`ans_name`** columns. You would do the following.

In [39]:
so.loc[so['viewcount'] > 20000, ['creationdate', 'viewcount', 'ans_name']].head(10)

,creationdate,viewcount,ans_name
6,2011-10-15 08:21:17,28159,Wes McKinney
8,2011-10-20 14:46:14,223746,Nick Crawford
14,2012-01-18 19:41:27,20614,Wes McKinney
31,2012-03-11 06:00:56,39323,huon
35,2012-03-18 12:53:06,43262,Wes McKinney
58,2012-04-04 23:17:23,20171,bmu
60,2012-04-08 18:01:13,77902,NaN
71,2012-04-18 03:59:55,88115,ely
75,2012-04-29 16:10:35,207980,Wes McKinney
76,2012-04-29 22:41:28,26110,andrew cooke


You could have broken each selection into pieces like this:

```Python

row_selection = so['viewcount'] > 20000
col_selection = ['creationdate', 'viewcount', 'ans_name']
so.loc[row_selection, col_selection]
```

## Lots of combinations possible with `.loc`
Remember that **`.loc`** can take a string, a list of strings or a slice. You can use all three possible ways to select your data. You can also make very complex boolean selections for your rows.

Let's select rows with **`favoritecount`** between 30 and 40 and every third column beginning from **`title`** to the end.

In [40]:
# weird but possible
so.loc[so['favoritecount'].between(30, 40), 'title'::3].head()

,title,favoritecount,ans_name
348,Pandas: create two new columns in a dataframe ...,39.0,DSM
555,How can I replace all the NaN values with Zero...,39.0,Aman
643,Pandas DataFrame: remove unwanted parts from s...,31.0,eumiro
667,"Creating an empty Pandas DataFrame, then filli...",40.0,Andy Hayden
892,Apply multiple functions to multiple groupby c...,35.0,Zelazny7


## Boolean selection for the columns?
It is actually possible to use a sequence of booleans to select columns. You pass a list, Series, or array of booleans the same length as the number of columns to **`.loc`**.

Let's do a simple manual example where we create a list of booleans by hand. First, let's find out how many columns are in our dataset

In [41]:
so.shape

(56398, 12)

Let's create a list of 12 booleans

In [42]:
col_bools = [True, False, False] * 4
col_bools

[True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False]

Use **`.loc`** to select all rows with just the `True` columns from **`col_bools`**.

In [43]:
so.loc[:, col_bools].head()

,id,viewcount,commentcount,quest_rep
0,5486226,2113,4,125.0
1,5515021,7015,6,3301.0
2,5558607,7392,0,3301.0
3,6467832,13056,0,117.0
4,7577546,2488,0,958.0


You can simultaneously select rows and columns too. Let's select the same columns but for rows that have over 500,000 views.

In [44]:
so.loc[so['viewcount'] > 500000, col_bools]

,id,viewcount,commentcount,quest_rep
171,11285613,526432,4,3369.0
181,11346283,931604,1,4206.0
397,12555323,698537,0,2920.0
581,13411544,802655,0,8807.0
1253,17071871,549481,0,3374.0
4587,19482970,541299,1,3483.0


## A more practical example
Let's see a slightly more practical example of doing boolean selection on the columns. Let's say we flipped 10 coins one-hundred times and store each trial in a column in the DataFrame below

In [45]:
coins = pd.DataFrame(np.random.randint(0, 2, (100, 10)), 
                     columns=list('abcdefghij'))
coins.head()

,a,b,c,d,e,f,g,h,i,j
0,0,0,0,1,0,1,0,0,1,0
1,1,0,0,1,1,0,0,1,0,0
2,0,1,1,0,1,1,0,1,0,0
3,1,0,1,0,0,1,1,1,0,0
4,0,0,0,1,0,1,0,0,1,1


In [46]:
coins.shape

(100, 10)

If we are interested in selecting only the columns that have more than 50% heads, we could first take the mean of each column like this.

In [47]:
coin_mean = coins.mean()
coin_mean

a    0.50
b    0.46
c    0.48
d    0.47
e    0.43
f    0.52
g    0.44
h    0.47
i    0.57
j    0.44
dtype: float64

Let's test the condition that the percentage is greater than .5

In [48]:
coin_mean > .5

a    False
b    False
c    False
d    False
e    False
f     True
g    False
h    False
i     True
j    False
dtype: bool

Finally, we can use this boolean Series to select only the columns that meet our criteria.

In [49]:
coins.loc[:, coins.mean() > .5].head()

,f,i
0,1,1
1,0,0
2,1,0
3,1,0
4,1,1


# Column to column comparisons
All of the previous Series comparisons happened against a single scalar value. It is possible to create a boolean Series by comparing one column to another. For instance, we can find all the questions where there are more answers than **`score`**.

In [50]:
criteria = so['answercount'] > so['score']
so[criteria].head()

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
10,8273092,2011-11-25 18:39:02,1,2333,python: pandas install errors,2,0,NaN,codingknob,2279.0,codingknob,2279.0
46,9927711,2012-03-29 14:42:42,1,1659,Reading csv in python pandas and handling bad ...,3,0,2.0,Dave31415,914.0,eumiro,104313.0
54,10003171,2012-04-03 23:59:41,1,404,What is an efficient way in pandas to do summa...,2,1,NaN,LmW.,486.0,Wes McKinney,43310.0
59,10027719,2012-04-05 11:28:00,0,500,Installing Pandas with Python 2.5 on Windows,1,0,NaN,JamesS,191.0,Wes McKinney,43310.0
77,10393447,2012-05-01 04:12:13,0,130,Scope gotcha when dynamically adding methods i...,2,0,NaN,Chris Billington,424.0,Ignacio Vazquez-Abrams,513959.0


In one line, the above would have looked like this:

```Python
so[so['answercount'] > so['score']]
```

## Almost never use `.iloc` with boolean selection
First, remember that **`.iloc`** uses INTEGER location to make its selections. 

You will rarely use **`.loc`** to do boolean selection and almost always use *just the indexing operator* or **`.loc`**. To see why, let's try and run a simple boolean selection to find all the rows that have more than 100,000 views.

In [51]:
so.iloc[so['viewcount'] > 100000]

NotImplementedError: iLocation based boolean indexing on an integer type is not available

## `NotImplementedError`
The pandas developers have not decided to boolean selection (with a Series) for **`.iloc`** so it does not work. You can however convert the Series to a list or a NumPy array as a workaround.

Let's save our Series to a variable and double-check its type.

In [52]:
criteria = so['viewcount'] > 100000
type(criteria)

pandas.core.series.Series

Let's grab the underlying NumPy array with the **`values`** attribute and pass it to **`.iloc`**

In [53]:
a = criteria.values
so.iloc[a].head()

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
8,7837722,2011-10-20 14:46:14,201,223746,What is the most efficient way to loop through...,8,3,115.0,Muppet,1563.0,Nick Crawford,2779.0
75,10373660,2012-04-29 16:10:35,199,207980,Converting a Pandas GroupBy object to DataFrame,5,0,90.0,saveenr,2421.0,Wes McKinney,43310.0
81,10457584,2012-05-05 00:00:12,72,117890,Redefining the Index in a Pandas DataFrame object,2,1,19.0,nitin,2945.0,Avaris,21067.0
100,10665889,2012-05-19 14:11:42,144,179896,How to take column-slices of dataframe in pandas,7,3,65.0,cpa,988.0,Ted Petrou,10426.0
106,10715965,2012-05-23 08:12:31,340,408347,add one row in a pandas.DataFrame,15,3,89.0,PhE,1988.0,fred,2342.0


You can make simultaneous column selection as well with integers.

In [54]:
so.iloc[a, [5, 10, 11]].head()

,answercount,ans_name,ans_rep
8,8,Nick Crawford,2779.0
75,5,Wes McKinney,43310.0
81,2,Avaris,21067.0
100,7,Ted Petrou,10426.0
106,15,fred,2342.0


I don't think I have ever used **`.iloc`** for boolean selection as its not implemented for Series. I added because it's one of the three main indexers in pandas and it's important to know that it's not used much at all for boolean selection.

## `.loc` and `[]` work the same on a Series for boolean selection
Boolean selection will work identically for **`.loc`** as it does with *just the indexing operator* on a Series. Both the indexers do row selection when passed a boolean Series. Since Series don't have columns, the two indexers are identical in this situation.

In [55]:
s = so['score']

In [56]:
s[s > 100].head()

8      201
17     136
75     199
100    144
106    340
Name: score, dtype: int64

In [57]:
s.loc[s > 100].head()

8      201
17     136
75     199
100    144
106    340
Name: score, dtype: int64

# Summary

* **Boolean Indexing** or **Boolean Selection** is the selection of a subset of a Series/DataFrame based on the values themselves and not the row/column labels or integer location
* Boolean selection is used to answer common queries like "find all the female engineers with a salary over 150k/year"
* To do boolean selection, you first create a sequence of True/False values and pass it to a DataFrame/Series indexer
* Each row of data is kept or discarded
* The indexing operators are **overloaded** - change functionality depending on what is passed to them
* Typically, you will first create a boolean Series with one of the 6 comparison operators
* You will pass this boolean series to one of the indexers to make your selection
* Use the **`isin`** method to test for multiple equalities in the same column
* Use **`isnull`** to find all rows with missing values in a particular column
* Can use the **`between`** Series method to test whether Series values are within a range
* You can create complex criteria with the **and** (**`&`**), **or** (**`|`**), and **not** (**`~`**) logical operators
* When you have multiple conditions in a single line, you must wrap each expression with a parentheses
* If you have complex criteria, think about storing each set of criteria into its own variable (i.e. don't do everything in one line)
* If you are only selecting rows, then you will almost always use *just the indexing operator*
* If you are simultaneously doing boolean selection on the rows and selecting column labels then you will use **`.loc`**
* You will almost never use **`.iloc`** to do boolean selection
* Boolean selection works the same for Series as it does for DataFrames

# More to the story
Believe or not, there is still more to the story when it comes to boolean selection. We will cover more advanced topics in part 7.

# Exercises
Boolean selection is difficult at first and the syntax is somewhat clunky. It will take some time to master. These questions will start easy and progressively become more difficult.

## Data for exercises
We will use two datasets for these exercises. The stack overflow dataset which you have seen above and the employee dataset.

In [58]:
so = pd.read_csv('../../data/stackoverflow_qa.csv')
so.head()

,id,creationdate,score,viewcount,title,answercount,commentcount,favoritecount,quest_name,quest_rep,ans_name,ans_rep
0,5486226,2011-03-30 12:26:50,4,2113,Rolling median in python,3,4,1.0,yueerhu,125.0,Mike Pennington,26995.0
1,5515021,2011-04-01 14:50:44,8,7015,Compute a compounded return series in Python,3,6,7.0,Jason Strimpel,3301.0,Mike Pennington,26995.0
2,5558607,2011-04-05 21:13:50,2,7392,Sort a pandas DataMatrix in ascending order,2,0,1.0,Jason Strimpel,3301.0,Wes McKinney,43310.0
3,6467832,2011-06-24 12:31:45,9,13056,How to get the correlation between two timeser...,1,0,7.0,user814005,117.0,Wes McKinney,43310.0
4,7577546,2011-09-28 01:58:38,9,2488,"Using pandas, how do I subsample a large DataF...",1,0,5.0,Uri Laserson,958.0,HYRY,54137.0


In [59]:
employee = pd.read_csv('../../data/employee.csv')
employee.head()

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE
0,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13
1,1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18
2,2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,Active,2015-02-03,2015-02-03
3,3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,Active,1982-02-08,1991-05-25
4,4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22


# Tip!
Append the **`head`** method at the end of your statements to prevent long output as was done above.

# Solutions
Make sure you check your answers with the [solutions notebook](./Solutions.ipynb).

### Exercise 1
<span  style="color:green; font-size:16px">Find all the questions that have exactly 5 answers</span>

In [60]:
# your code here

### Exercise 2
<span  style="color:green; font-size:16px">Find all the questions that have less than 10 views</span>

In [61]:
# your code here

### Exercise 3
<span  style="color:green; font-size:16px">Find all the questions where the person asking it is the same as the person answering it</span>

In [62]:
# your code here

### Exercise 4
<span  style="color:green; font-size:16px">Find all the questions that don't have an accepted answer, but have a score of more than 100</span>

In [63]:
# your code here

### Exercise 5
<span  style="color:green; font-size:16px">Find all the questions where the reputation of the person asking the question is higher than the person answering it. Then find the percentage of times this happens</span>

In [64]:
# your code here

### Exercise 6
<span  style="color:green; font-size:16px">Find all the questions where the number of answers is between 5 and 10 inclusive, and the number of views is less than 1,000.</span>

In [65]:
# your code here

### Exercise 7
<span  style="color:green; font-size:16px">Find the inverse of exercise 6. Verify your results by adding the rows of both returned Series to see if it matches the number of rows of the original</span>

In [66]:
# your code here

### Use the employee data for the rest of the exercises

In [67]:
employee = pd.read_csv('../../data/employee.csv')
employee.head()

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE
0,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13
1,1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18
2,2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,Active,2015-02-03,2015-02-03
3,3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,Active,1982-02-08,1991-05-25
4,4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22


### Exercise 8
<span  style="color:green; font-size:16px">Find all the **`Black or African American`** females that work in the **`Houston Police Department-HPD`**</span>

In [68]:
# your code here

### Exercise 9
<span  style="color:green; font-size:16px">Find the females that have a salary over 100,000 OR males with salary under 50,000</span>

In [69]:
# your code here

### Exercise 10
<span  style="color:green; font-size:16px">Find the females in the following departments with salary over 60,000 (Parks & Recreation, Solid Waste Management, Fleet Management Department, Library)  </span>

In [70]:
# your code here

### Exercise 11
<span  style="color:green; font-size:16px">Find all the males with salary over 100,000. Return only the race, gender and salary columns</span>

In [71]:
# your code here

### Exercise 12
<span  style="color:green; font-size:16px">Select all salaries as a Series in a separate variable. From this series select all salaries under 25,000</span>

In [72]:
# your code here

### Exercise 13
<span  style="color:green; font-size:16px">Get the same exact result as exercise 11, but make your selection from the employee DataFrame. Use only a single line of code</span>

In [73]:
# your code here